In [1]:
datasets= ["mouse_kidney_cell","human_pbmc_cell","human_pbmc2_cell","human_skin_cell","human_lung_cell","mouse_kidney_drop","mouse_kidney_cl2","mouse_kidney_10x"]

In [2]:
import argparse

import numpy as np

from dance.datasets.singlemodality import ClusteringDataset
from dance.modules.single_modality.clustering.graphsc import GraphSC
from dance.utils import set_seed


parser = argparse.ArgumentParser()
parser.add_argument("-e", "--epochs", default=100, type=int)
parser.add_argument("-dv", "--device", default="cpu")
parser.add_argument("-if", "--in_feats", default=50, type=int)
parser.add_argument("-bs", "--batch_size", default=128, type=int)
parser.add_argument("-nw", "--normalize_weights", default="log_per_cell", choices=["log_per_cell", "per_cell"])
parser.add_argument("-ac", "--activation", default="relu", choices=["leaky_relu", "relu", "prelu", "gelu"])
parser.add_argument("-drop", "--dropout", default=0.1, type=float)
parser.add_argument("-nf", "--node_features", default="scale", choices=["scale_by_cell", "scale", "none"])
parser.add_argument("-sev", "--same_edge_values", default=False, action="store_true")
parser.add_argument("-en", "--edge_norm", default=True, action="store_true")
parser.add_argument("-hr", "--hidden_relu", default=False, action="store_true")
parser.add_argument("-hbn", "--hidden_bn", default=False, action="store_true")
parser.add_argument("-lr", "--learning_rate", type=float, default=1e-5)
parser.add_argument("-nl", "--n_layers", type=int, default=1, choices=[1, 2])
parser.add_argument("-agg", "--agg", default="sum", choices=["sum", "mean"])
parser.add_argument("-hd", "--hidden_dim", type=int, default=200)
parser.add_argument("-nh", "--n_hidden", type=int, default=1, choices=[0, 1, 2])
parser.add_argument("-h1", "--hidden_1", type=int, default=300)
parser.add_argument("-h2", "--hidden_2", type=int, default=0)
parser.add_argument("-ng", "--nb_genes", type=int, default=3000)
parser.add_argument("-nr", "--num_run", type=int, default=1)
parser.add_argument("-nbw", "--num_workers", type=int, default=1)
parser.add_argument("-eve", "--eval_epoch", action="store_true")
parser.add_argument("-show", "--show_epoch_ari", action="store_true")
parser.add_argument("-plot", "--plot", default=False, action="store_true")
parser.add_argument("-dd", "--data_dir", default="./data", type=str)
parser.add_argument("-data", "--dataset", default="10X_PBMC",
                    choices=["10X_PBMC", "mouse_bladder_cell", "mouse_ES_cell", "worm_neuron_cell","mouse_kidney_cell","human_pbmc_cell","human_pbmc2_cell","human_skin_cell","human_lung_cell","mouse_kidney_drop","mouse_kidney_cl2","mouse_kidney_10x"])
parser.add_argument("--seed", type=int, default=42)
parser.add_argument("--cache", action="store_true", help="Cache processed data.")
graphsc_scores=[]
for dataset in datasets:
    args = parser.parse_args(args=['--dataset',dataset])
    set_seed(args.seed)

    # Load data and perform necessary preprocessing
    dataloader = ClusteringDataset(args.data_dir, args.dataset)
    preprocessing_pipeline = GraphSC.preprocessing_pipeline(
        n_top_genes=args.nb_genes,
        normalize_weights=args.normalize_weights,
        n_components=args.in_feats,
        normalize_edges=args.edge_norm,
    )
    data = dataloader.load_data(transform=preprocessing_pipeline, cache=args.cache)

    graph, y = data.get_train_data()
    n_clusters = len(np.unique(y))

    # Evaluate model for several runs
    for run in range(args.num_run):
        set_seed(args.seed + run)
        model = GraphSC(agg=args.agg, activation=args.activation, in_feats=args.in_feats, n_hidden=args.n_hidden,
                        hidden_dim=args.hidden_dim, hidden_1=args.hidden_1, hidden_2=args.hidden_2,
                        dropout=args.dropout, n_layers=args.n_layers, hidden_relu=args.hidden_relu,
                        hidden_bn=args.hidden_bn, n_clusters=n_clusters, cluster_method="leiden",
                        num_workers=args.num_workers, device=args.device)
        model.fit(graph, epochs=args.epochs, lr=args.learning_rate, show_epoch_ari=args.show_epoch_ari,
                  eval_epoch=args.eval_epoch)
        score = model.score(None, y)
        print(f"{score=:.4f}")
        graphsc_scores.append(score)
""" Reproduction information
10X PBMC:
python graphsc.py --dataset 10X_PBMC

Mouse ES:
python graphsc.py --dataset mouse_ES_cell

Worm Neuron:
python graphsc.py --dataset worm_neuron_cell

Mouse Bladder:
python graphsc.py --dataset mouse_bladder_cell
"""


[INFO][2023-09-02 15:44:15,069][dance][set_seed] Setting global random seed to 42
[INFO][2023-09-02 15:44:15,235][dance][load_data] Raw data loaded:
Data object that wraps (.data):
AnnData object with n_obs × n_vars = 902 × 16468
    uns: 'dance_config'
    obsm: 'Group'
[INFO][2023-09-02 15:44:15,236][dance.Compose][__call__] Applying composed transformations:
Compose(
  AnnDataTransform(func=scanpy.preprocessing._simple.filter_genes, func_kwargs={'min_counts': 3}),
  AnnDataTransform(func=scanpy.preprocessing._simple.filter_cells, func_kwargs={'min_counts': 1}),
  AnnDataTransform(func=scanpy.preprocessing._normalization.normalize_total, func_kwargs={}),
  AnnDataTransform(func=scanpy.preprocessing._simple.log1p, func_kwargs={}),
  AnnDataTransform(func=scanpy.preprocessing._highly_variable_genes.highly_variable_genes, func_kwargs={'min_mean': 0.0125, 'max_mean': 4, 'flavor': 'cell_ranger', 'min_disp': 0.5, 'n_top_genes': 3000, 'subset': True}),
  AnnDataTransform(func=scanpy.preproc

[INFO][2023-09-02 15:44:15,588][dance.WeightedFeaturePCA][__call__] Normalizing feature before PCA decomposition with mode=standardize and axis=0
[INFO][2023-09-02 15:44:15,597][dance.WeightedFeaturePCA][__call__] Start decomposing None features (902, 3000) (k=50)
[INFO][2023-09-02 15:44:15,636][dance.WeightedFeaturePCA][__call__] Total explained variance: 47.53%
[INFO][2023-09-02 15:44:15,718][dance.CellFeatureGraph][__call__] Number of nonzero entries: 1,423,104
[INFO][2023-09-02 15:44:15,719][dance.CellFeatureGraph][__call__] Nonzero rate = 52.6%
[INFO][2023-09-02 15:44:16,445][dance.SetConfig][__call__] Updating the dance data object config options:
{'feature_channel': 'CellFeatureGraph',
 'feature_channel_type': 'uns',
 'label_channel': 'Group'}
[INFO][2023-09-02 15:44:16,445][dance][set_config_from_dict] Setting config 'feature_channel' to 'CellFeatureGraph'
[INFO][2023-09-02 15:44:16,446][dance][set_config_from_dict] Setting config 'feature_channel_type' to 'uns'
[INFO][2023-09-

score=0.4672


[INFO][2023-09-02 15:46:19,733][dance][load_data] Raw data loaded:
Data object that wraps (.data):
AnnData object with n_obs × n_vars = 648 × 64535
    uns: 'dance_config'
    obsm: 'Group'
[INFO][2023-09-02 15:46:19,734][dance.Compose][__call__] Applying composed transformations:
Compose(
  AnnDataTransform(func=scanpy.preprocessing._simple.filter_genes, func_kwargs={'min_counts': 3}),
  AnnDataTransform(func=scanpy.preprocessing._simple.filter_cells, func_kwargs={'min_counts': 1}),
  AnnDataTransform(func=scanpy.preprocessing._normalization.normalize_total, func_kwargs={}),
  AnnDataTransform(func=scanpy.preprocessing._simple.log1p, func_kwargs={}),
  AnnDataTransform(func=scanpy.preprocessing._highly_variable_genes.highly_variable_genes, func_kwargs={'min_mean': 0.0125, 'max_mean': 4, 'flavor': 'cell_ranger', 'min_disp': 0.5, 'n_top_genes': 3000, 'subset': True}),
  AnnDataTransform(func=scanpy.preprocessing._simple.log1p, func_kwargs={}),
  AnnDataTransform(func=scanpy.preprocessin

[INFO][2023-09-02 15:46:20,202][dance.WeightedFeaturePCA][__call__] Normalizing feature before PCA decomposition with mode=standardize and axis=0
[INFO][2023-09-02 15:46:20,207][dance.WeightedFeaturePCA][__call__] Start decomposing None features (648, 3000) (k=50)
[INFO][2023-09-02 15:46:20,233][dance.WeightedFeaturePCA][__call__] Total explained variance: 17.32%
[INFO][2023-09-02 15:46:20,253][dance.CellFeatureGraph][__call__] Number of nonzero entries: 114,062
[INFO][2023-09-02 15:46:20,254][dance.CellFeatureGraph][__call__] Nonzero rate = 5.9%
[INFO][2023-09-02 15:46:20,820][dance.SetConfig][__call__] Updating the dance data object config options:
{'feature_channel': 'CellFeatureGraph',
 'feature_channel_type': 'uns',
 'label_channel': 'Group'}
[INFO][2023-09-02 15:46:20,820][dance][set_config_from_dict] Setting config 'feature_channel' to 'CellFeatureGraph'
[INFO][2023-09-02 15:46:20,821][dance][set_config_from_dict] Setting config 'feature_channel_type' to 'uns'
[INFO][2023-09-02 

...


[INFO][2023-09-02 15:58:12,368][dance][set_seed] Setting global random seed to 42
[INFO][2023-09-02 15:58:12,500][dance][load_data] Raw data loaded:
Data object that wraps (.data):
AnnData object with n_obs × n_vars = 902 × 16468
    uns: 'dance_config'
    obsm: 'Group'
[INFO][2023-09-02 15:58:12,500][dance.Compose][__call__] Applying composed transformations:
Compose(
  AnnDataTransform(func=scanpy.preprocessing._simple.filter_genes, func_kwargs={'min_counts': 3}),
  AnnDataTransform(func=scanpy.preprocessing._simple.filter_cells, func_kwargs={'min_counts': 1}),
  AnnDataTransform(func=scanpy.preprocessing._normalization.normalize_total, func_kwargs={}),
  AnnDataTransform(func=scanpy.preprocessing._simple.log1p, func_kwargs={}),
  AnnDataTransform(func=scanpy.preprocessing._highly_variable_genes.highly_variable_genes, func_kwargs={'min_mean': 0.0125, 'max_mean': 4, 'flavor': 'cell_ranger', 'min_disp': 0.5, 'n_top_genes': 3000, 'subset': True}),
  AnnDataTransform(func=scanpy.prepro

score=0.9097


[INFO][2023-09-02 15:58:12,974][dance.WeightedFeaturePCA][__call__] Normalizing feature before PCA decomposition with mode=standardize and axis=0
[INFO][2023-09-02 15:58:12,983][dance.WeightedFeaturePCA][__call__] Start decomposing None features (902, 3000) (k=50)
[INFO][2023-09-02 15:58:13,018][dance.WeightedFeaturePCA][__call__] Total explained variance: 47.53%
[INFO][2023-09-02 15:58:13,079][dance.CellFeatureGraph][__call__] Number of nonzero entries: 1,423,104
[INFO][2023-09-02 15:58:13,080][dance.CellFeatureGraph][__call__] Nonzero rate = 52.6%
[INFO][2023-09-02 15:58:13,786][dance.SetConfig][__call__] Updating the dance data object config options:
{'feature_channel': 'CellFeatureGraph',
 'feature_channel_type': 'uns',
 'label_channel': 'Group'}
[INFO][2023-09-02 15:58:13,787][dance][set_config_from_dict] Setting config 'feature_channel' to 'CellFeatureGraph'
[INFO][2023-09-02 15:58:13,787][dance][set_config_from_dict] Setting config 'feature_channel_type' to 'uns'
[INFO][2023-09-

score=0.4672


' Reproduction information\n10X PBMC:\npython graphsc.py --dataset 10X_PBMC\n\nMouse ES:\npython graphsc.py --dataset mouse_ES_cell\n\nWorm Neuron:\npython graphsc.py --dataset worm_neuron_cell\n\nMouse Bladder:\npython graphsc.py --dataset mouse_bladder_cell\n'

In [3]:
graphsc_scores

[0.4672362410138263,
 0.5968663719240431,
 0.24416902238895835,
 0.49769852192025643,
 0.5912071617671083,
 0.9097075531594662,
 0.9097075531594662,
 0.4672362410138263]

In [4]:
import argparse
import os

import numpy as np

from dance.datasets.singlemodality import ClusteringDataset
from dance.modules.single_modality.clustering.scdcc import ScDCC
from dance.transforms.preprocess import generate_random_pair
from dance.utils import set_seed

parser = argparse.ArgumentParser(description="train", formatter_class=argparse.ArgumentDefaultsHelpFormatter)
parser.add_argument("--label_cells", default=0.1, type=float)
parser.add_argument("--label_cells_files", default="label_mouse_ES_cell.txt")
parser.add_argument("--n_pairwise", default=0, type=int)
parser.add_argument("--n_pairwise_error", default=0, type=float)
parser.add_argument("--batch_size", default=256, type=int)
parser.add_argument("--data_dir", default="./data")
parser.add_argument("--dataset", default="mouse_ES_cell", type=str,
                    choices=["10X_PBMC", "mouse_bladder_cell", "mouse_ES_cell", "worm_neuron_cell","mouse_kidney_cell","human_pbmc_cell","human_pbmc2_cell","human_skin_cell","human_lung_cell","mouse_kidney_drop","mouse_kidney_cl2","mouse_kidney_10x"])
parser.add_argument("--epochs", default=500, type=int)
parser.add_argument("--pretrain_epochs", default=50, type=int)
parser.add_argument("--lr", default=0.01, type=float)
parser.add_argument("--pretrain_lr", default=0.001, type=float)
parser.add_argument("--sigma", default=2.5, type=float, help="coefficient of Gaussian noise")
parser.add_argument("--gamma", default=1., type=float, help="coefficient of clustering loss")
parser.add_argument("--ml_weight", default=1., type=float, help="coefficient of must-link loss")
parser.add_argument("--cl_weight", default=1., type=float, help="coefficient of cannot-link loss")
parser.add_argument("--update_interval", default=1, type=int)
parser.add_argument("--tol", default=0.00001, type=float)
parser.add_argument("--ae_weights", default=None)
parser.add_argument("--ae_weight_file", default="AE_weights.pth.tar")
parser.add_argument("--device", default="cpu")
parser.add_argument("--seed", type=int, default=42)
parser.add_argument("--cache", action="store_true", help="Cache processed data.")
scdcc_scores=[]
for dataset in datasets:
    args = parser.parse_args(['--dataset',dataset,'--label_cells_files','label_mouse_kidney.txt','--gamma','1.5'])
    set_seed(args.seed)

    # Load data and perform necessary preprocessing
    dataloader = ClusteringDataset(args.data_dir, args.dataset)
    preprocessing_pipeline = ScDCC.preprocessing_pipeline()
    data = dataloader.load_data(transform=preprocessing_pipeline, cache=args.cache)

    # inputs: x, x_raw, n_counts
    inputs, y = data.get_train_data()
    n_clusters = len(np.unique(y))
    in_dim = inputs[0].shape[1]

    # Generate random pairs
    if not os.path.exists(args.label_cells_files):
        indx = np.arange(len(y))
        np.random.shuffle(indx)
        label_cell_indx = indx[0:int(np.ceil(args.label_cells * len(y)))]
    else:
        label_cell_indx = np.loadtxt(args.label_cells_files, dtype=np.int)

    if args.n_pairwise > 0:
        ml_ind1, ml_ind2, cl_ind1, cl_ind2, error_num = generate_random_pair(y, label_cell_indx, args.n_pairwise,
                                                                             args.n_pairwise_error)
        print("Must link paris: %d" % ml_ind1.shape[0])
        print("Cannot link paris: %d" % cl_ind1.shape[0])
        print("Number of error pairs: %d" % error_num)
    else:
        ml_ind1, ml_ind2, cl_ind1, cl_ind2 = np.array([]), np.array([]), np.array([]), np.array([])

    # Build and train moodel
    model = ScDCC(input_dim=in_dim, z_dim=32, n_clusters=n_clusters, encodeLayer=[256, 64], decodeLayer=[64, 256],
                  sigma=args.sigma, gamma=args.gamma, ml_weight=args.ml_weight, cl_weight=args.ml_weight,
                  device=args.device, pretrain_path=f"scdcc_{args.dataset}_pre.pkl")
    model.fit(inputs, y, lr=args.lr, batch_size=args.batch_size, epochs=args.epochs, ml_ind1=ml_ind1, ml_ind2=ml_ind2,
              cl_ind1=cl_ind1, cl_ind2=cl_ind2, update_interval=args.update_interval, tol=args.tol,
              pt_batch_size=args.batch_size, pt_lr=args.pretrain_lr, pt_epochs=args.pretrain_epochs)

    # Evaluate model predictions
    score = model.score(None, y)
    print(f"{score=:.4f}")
    scdcc_scores.append(score)
""" Reproduction information
10X PBMC:
python scdcc.py --dataset 10X_PBMC --label_cells_files label_10X_PBMC.txt --gamma=1.5

Mouse ES:
python scdcc.py --dataset mouse_ES_cell --label_cells_files label_mouse_ES_cell.txt --gamma 1 --ml_weight 0.8 --cl_weight 0.8

Worm Neuron:
python scdcc.py --dataset worm_neuron_cell --label_cells_files label_worm_neuron_cell.txt --gamma 1 --pretrain_epochs 300

Mouse Bladder:
python scdcc.py --dataset mouse_bladder_cell --label_cells_files label_mouse_bladder_cell.txt --gamma 1.5 --pretrain_epochs 100 --sigma 3
"""


[INFO][2023-09-02 16:00:11,499][dance][set_seed] Setting global random seed to 42
[INFO][2023-09-02 16:00:11,625][dance][load_data] Raw data loaded:
Data object that wraps (.data):
AnnData object with n_obs × n_vars = 902 × 16468
    uns: 'dance_config'
    obsm: 'Group'
[INFO][2023-09-02 16:00:11,626][dance.Compose][__call__] Applying composed transformations:
Compose(
  AnnDataTransform(func=scanpy.preprocessing._simple.filter_genes, func_kwargs={'min_counts': 1}),
  AnnDataTransform(func=scanpy.preprocessing._simple.filter_cells, func_kwargs={'min_counts': 1}),
  SaveRaw(),
  AnnDataTransform(func=scanpy.preprocessing._normalization.normalize_total, func_kwargs={}),
  AnnDataTransform(func=scanpy.preprocessing._simple.log1p, func_kwargs={}),
  AnnDataTransform(func=scanpy.preprocessing._simple.scale, func_kwargs={}),
  SetConfig(config_dict={'feature_channel': [None, None, 'n_counts'], 'feature_channel_type': ['X', 'raw_X', 'obs'], 'label_channel': 'Group'}),
)
[INFO][2023-09-02 16:

score=0.9965


[INFO][2023-09-02 16:00:13,466][dance][load_data] Raw data loaded:
Data object that wraps (.data):
AnnData object with n_obs × n_vars = 648 × 64535
    uns: 'dance_config'
    obsm: 'Group'
[INFO][2023-09-02 16:00:13,467][dance.Compose][__call__] Applying composed transformations:
Compose(
  AnnDataTransform(func=scanpy.preprocessing._simple.filter_genes, func_kwargs={'min_counts': 1}),
  AnnDataTransform(func=scanpy.preprocessing._simple.filter_cells, func_kwargs={'min_counts': 1}),
  SaveRaw(),
  AnnDataTransform(func=scanpy.preprocessing._normalization.normalize_total, func_kwargs={}),
  AnnDataTransform(func=scanpy.preprocessing._simple.log1p, func_kwargs={}),
  AnnDataTransform(func=scanpy.preprocessing._simple.scale, func_kwargs={}),
  SetConfig(config_dict={'feature_channel': [None, None, 'n_counts'], 'feature_channel_type': ['X', 'raw_X', 'obs'], 'label_channel': 'Group'}),
)
[INFO][2023-09-02 16:00:13,660][dance.SaveRaw][__call__] Saving data to ``.raw``
[INFO][2023-09-02 16:0

score=0.2302


[INFO][2023-09-02 16:00:17,046][dance][fit] #Epoch   1: Total: 0.9746, Clustering Loss: 0.2618, ZINB Loss: 0.7129
[INFO][2023-09-02 16:00:17,599][dance][fit] #Epoch   2: Total: 0.9754, Clustering Loss: 0.2626, ZINB Loss: 0.7129
[INFO][2023-09-02 16:00:18,151][dance][fit] #Epoch   3: Total: 0.9743, Clustering Loss: 0.2623, ZINB Loss: 0.7120
[INFO][2023-09-02 16:00:18,719][dance][fit] #Epoch   4: Total: 0.9751, Clustering Loss: 0.2618, ZINB Loss: 0.7133
[INFO][2023-09-02 16:00:19,273][dance][fit] #Epoch   5: Total: 0.9747, Clustering Loss: 0.2610, ZINB Loss: 0.7136
[INFO][2023-09-02 16:00:19,825][dance][fit] #Epoch   6: Total: 0.9739, Clustering Loss: 0.2602, ZINB Loss: 0.7136
[INFO][2023-09-02 16:00:20,377][dance][fit] #Epoch   7: Total: 0.9722, Clustering Loss: 0.2593, ZINB Loss: 0.7130
[INFO][2023-09-02 16:00:20,927][dance][fit] #Epoch   8: Total: 0.9721, Clustering Loss: 0.2582, ZINB Loss: 0.7139
[INFO][2023-09-02 16:00:21,479][dance][fit] #Epoch   9: Total: 0.9707, Clustering Loss: 

...

[INFO][2023-09-02 16:00:32,671][dance][fit] #Epoch   1: Total: 0.7840, Clustering Loss: 0.1484, ZINB Loss: 0.6356
[INFO][2023-09-02 16:00:32,702][dance][fit] Reach tolerance threshold (0.000e+00 < 1.000e-05). Stopping training.
[INFO][2023-09-02 16:00:32,704][dance][set_seed] Setting global random seed to 42
[INFO][2023-09-02 16:00:32,780][dance][load_data] Raw data loaded:
Data object that wraps (.data):
AnnData object with n_obs × n_vars = 902 × 16468
    uns: 'dance_config'
    obsm: 'Group'
[INFO][2023-09-02 16:00:32,780][dance.Compose][__call__] Applying composed transformations:
Compose(
  AnnDataTransform(func=scanpy.preprocessing._simple.filter_genes, func_kwargs={'min_counts': 1}),
  AnnDataTransform(func=scanpy.preprocessing._simple.filter_cells, func_kwargs={'min_counts': 1}),
  SaveRaw(),
  AnnDataTransform(func=scanpy.preprocessing._normalization.normalize_total, func_kwargs={}),
  AnnDataTransform(func=scanpy.preprocessing._simple.log1p, func_kwargs={}),
  AnnDataTransfor

score=0.8688


[INFO][2023-09-02 16:00:32,957][dance.SetConfig][__call__] Updating the dance data object config options:
{'feature_channel': [None, None, 'n_counts'],
 'feature_channel_type': ['X', 'raw_X', 'obs'],
 'label_channel': 'Group'}
[INFO][2023-09-02 16:00:32,957][dance][set_config_from_dict] Setting config 'feature_channel' to [None, None, 'n_counts']
[INFO][2023-09-02 16:00:32,958][dance][set_config_from_dict] Setting config 'feature_channel_type' to ['X', 'raw_X', 'obs']
[INFO][2023-09-02 16:00:32,959][dance][set_config_from_dict] Setting config 'label_channel' to 'Group'
[INFO][2023-09-02 16:00:32,960][dance][load_data] Data transformed:
Data object that wraps (.data):
AnnData object with n_obs × n_vars = 902 × 16468
    obs: 'n_counts'
    var: 'n_counts', 'mean', 'std'
    uns: 'dance_config', 'log1p'
    obsm: 'Group'
[INFO][2023-09-02 16:00:32,960][dance][wrapped_func] Took 0:00:00.255337 to load and process data.
[INFO][2023-09-02 16:00:33,057][dance][_pretrain] Loading pre-trained 

score=1.0000


' Reproduction information\n10X PBMC:\npython scdcc.py --dataset 10X_PBMC --label_cells_files label_10X_PBMC.txt --gamma=1.5\n\nMouse ES:\npython scdcc.py --dataset mouse_ES_cell --label_cells_files label_mouse_ES_cell.txt --gamma 1 --ml_weight 0.8 --cl_weight 0.8\n\nWorm Neuron:\npython scdcc.py --dataset worm_neuron_cell --label_cells_files label_worm_neuron_cell.txt --gamma 1 --pretrain_epochs 300\n\nMouse Bladder:\npython scdcc.py --dataset mouse_bladder_cell --label_cells_files label_mouse_bladder_cell.txt --gamma 1.5 --pretrain_epochs 100 --sigma 3\n'

In [5]:
scdcc_scores

[0.9965362702578435,
 0.2302306982490099,
 0.5338897606315373,
 0.6257754637662535,
 0.7682366319440681,
 0.8688432600660997,
 0.8688432600660997,
 1.0]

In [6]:
import argparse

import numpy as np

from dance.datasets.singlemodality import ClusteringDataset
from dance.modules.single_modality.clustering.scdeepcluster import ScDeepCluster
from dance.utils import set_seed


parser = argparse.ArgumentParser(description="train", formatter_class=argparse.ArgumentDefaultsHelpFormatter)
parser.add_argument("--knn", default=20, type=int,
                    help="number of nearest neighbors, used by the Louvain algorithm")
parser.add_argument(
    "--resolution", default=.8, type=float,
    help="resolution parameter, used by the Louvain algorithm, larger value for more number of clusters")
parser.add_argument("--select_genes", default=0, type=int, help="number of selected genes, 0 means using all genes")
parser.add_argument("--batch_size", default=256, type=int)
parser.add_argument("--data_dir", default="./data")
parser.add_argument("--dataset", default="mouse_bladder_cell", type=str,
                    choices=["10X_PBMC", "mouse_bladder_cell", "mouse_ES_cell", "worm_neuron_cell","mouse_kidney_cell","human_pbmc_cell","human_pbmc2_cell","human_skin_cell","human_lung_cell","mouse_kidney_drop","mouse_kidney_cl2","mouse_kidney_10x"])
parser.add_argument("--epochs", default=500, type=int)
parser.add_argument("--pretrain_epochs", default=50, type=int)
parser.add_argument("--lr", default=0.1, type=float)
parser.add_argument("--pretrain_lr", default=0.001, type=float)
parser.add_argument("--gamma", default=1., type=float, help="coefficient of clustering loss")
parser.add_argument("--sigma", default=2.5, type=float, help="coefficient of random noise")
parser.add_argument("--update_interval", default=1, type=int)
parser.add_argument("--tol", default=0.001, type=float,
                    help="tolerance for delta clustering labels to terminate training stage")
parser.add_argument("--ae_weights", default=None, help="file to pretrained weights, None for a new pretraining")
parser.add_argument("--device", default="cpu")
parser.add_argument("--seed", type=int, default=42)
parser.add_argument("--cache", action="store_true", help="Cache processed data.")
scdeepcluster_scores=[]
for dataset in datasets:
    args = parser.parse_args(args=['--dataset',dataset])
    set_seed(args.seed)

    # Load data and perform necessary preprocessing
    dataloader = ClusteringDataset(args.data_dir, args.dataset)
    preprocessing_pipeline = ScDeepCluster.preprocessing_pipeline()
    data = dataloader.load_data(transform=preprocessing_pipeline, cache=args.cache)

    # inputs: x, x_raw, n_counts
    inputs, y = data.get_train_data()
    n_clusters = len(np.unique(y))
    in_dim = inputs[0].shape[1]

    # Build and train model
    model = ScDeepCluster(input_dim=in_dim, z_dim=32, encodeLayer=[256, 64], decodeLayer=[64, 256], sigma=args.sigma,
                          gamma=args.gamma, device=args.device, pretrain_path=f"scdeepcluster_{args.dataset}_pre.pkl")
    model.fit(inputs, y, n_clusters=n_clusters, y_pred_init=None, lr=args.lr, batch_size=args.batch_size,
              epochs=args.epochs, update_interval=args.update_interval, tol=args.tol, pt_batch_size=args.batch_size,
              pt_lr=args.pretrain_lr, pt_epochs=args.pretrain_epochs)

    # Evaluate model predictions
    score = model.score(None, y)
    print(f"{score=:.4f}")
    scdeepcluster_scores.append(score)
""" Reproduction information
10X PBMC:
python scdeepcluster.py --dataset 10X_PBMC

Mouse ES:
python scdeepcluster.py --dataset mouse_ES_cell

Worm Neuron:
python scdeepcluster.py --dataset worm_neuron_cell --pretrain_epochs 300

Mouse Bladder:
python scdeepcluster.py --dataset mouse_bladder_cell --pretrain_epochs 300 --sigma 2.75
"""


[INFO][2023-09-02 16:00:33,931][dance][set_seed] Setting global random seed to 42


[INFO][2023-09-02 16:00:34,055][dance][load_data] Raw data loaded:
Data object that wraps (.data):
AnnData object with n_obs × n_vars = 902 × 16468
    uns: 'dance_config'
    obsm: 'Group'
[INFO][2023-09-02 16:00:34,056][dance.Compose][__call__] Applying composed transformations:
Compose(
  AnnDataTransform(func=scanpy.preprocessing._simple.filter_genes, func_kwargs={'min_counts': 1}),
  AnnDataTransform(func=scanpy.preprocessing._simple.filter_cells, func_kwargs={'min_counts': 1}),
  SaveRaw(),
  AnnDataTransform(func=scanpy.preprocessing._normalization.normalize_total, func_kwargs={}),
  AnnDataTransform(func=scanpy.preprocessing._simple.log1p, func_kwargs={}),
  AnnDataTransform(func=scanpy.preprocessing._simple.scale, func_kwargs={}),
  SetConfig(config_dict={'feature_channel': [None, None, 'n_counts'], 'feature_channel_type': ['X', 'raw_X', 'obs'], 'label_channel': 'Group'}),
)
[INFO][2023-09-02 16:00:34,179][dance.SaveRaw][__call__] Saving data to ``.raw``
[INFO][2023-09-02 16:0

score=0.9965


[INFO][2023-09-02 16:00:35,451][dance][load_data] Raw data loaded:
Data object that wraps (.data):
AnnData object with n_obs × n_vars = 648 × 64535
    uns: 'dance_config'
    obsm: 'Group'
[INFO][2023-09-02 16:00:35,452][dance.Compose][__call__] Applying composed transformations:
Compose(
  AnnDataTransform(func=scanpy.preprocessing._simple.filter_genes, func_kwargs={'min_counts': 1}),
  AnnDataTransform(func=scanpy.preprocessing._simple.filter_cells, func_kwargs={'min_counts': 1}),
  SaveRaw(),
  AnnDataTransform(func=scanpy.preprocessing._normalization.normalize_total, func_kwargs={}),
  AnnDataTransform(func=scanpy.preprocessing._simple.log1p, func_kwargs={}),
  AnnDataTransform(func=scanpy.preprocessing._simple.scale, func_kwargs={}),
  SetConfig(config_dict={'feature_channel': [None, None, 'n_counts'], 'feature_channel_type': ['X', 'raw_X', 'obs'], 'label_channel': 'Group'}),
)
[INFO][2023-09-02 16:00:35,638][dance.SaveRaw][__call__] Saving data to ``.raw``
[INFO][2023-09-02 16:0

score=0.2403


...

[INFO][2023-09-02 16:00:56,491][dance][fit] Epoch   1: Total: 0.73605788, Clustering Loss: 0.09888156, ZINB Loss: 0.63717633
[INFO][2023-09-02 16:00:56,520][dance][fit] Reach tolerance threshold (0.000e+00 < 1.000e-03). Stopping training.
[INFO][2023-09-02 16:00:56,523][dance][set_seed] Setting global random seed to 42
[INFO][2023-09-02 16:00:56,621][dance][load_data] Raw data loaded:
Data object that wraps (.data):
AnnData object with n_obs × n_vars = 902 × 16468
    uns: 'dance_config'
    obsm: 'Group'
[INFO][2023-09-02 16:00:56,621][dance.Compose][__call__] Applying composed transformations:
Compose(
  AnnDataTransform(func=scanpy.preprocessing._simple.filter_genes, func_kwargs={'min_counts': 1}),
  AnnDataTransform(func=scanpy.preprocessing._simple.filter_cells, func_kwargs={'min_counts': 1}),
  SaveRaw(),
  AnnDataTransform(func=scanpy.preprocessing._normalization.normalize_total, func_kwargs={}),
  AnnDataTransform(func=scanpy.preprocessing._simple.log1p, func_kwargs={}),
  AnnD

score=0.8565


[INFO][2023-09-02 16:00:56,750][dance.SaveRaw][__call__] Saving data to ``.raw``
[INFO][2023-09-02 16:00:56,840][dance.SetConfig][__call__] Updating the dance data object config options:
{'feature_channel': [None, None, 'n_counts'],
 'feature_channel_type': ['X', 'raw_X', 'obs'],
 'label_channel': 'Group'}
[INFO][2023-09-02 16:00:56,841][dance][set_config_from_dict] Setting config 'feature_channel' to [None, None, 'n_counts']
[INFO][2023-09-02 16:00:56,842][dance][set_config_from_dict] Setting config 'feature_channel_type' to ['X', 'raw_X', 'obs']
[INFO][2023-09-02 16:00:56,843][dance][set_config_from_dict] Setting config 'label_channel' to 'Group'
[INFO][2023-09-02 16:00:56,843][dance][load_data] Data transformed:
Data object that wraps (.data):
AnnData object with n_obs × n_vars = 902 × 16468
    obs: 'n_counts'
    var: 'n_counts', 'mean', 'std'
    uns: 'dance_config', 'log1p'
    obsm: 'Group'
[INFO][2023-09-02 16:00:56,844][dance][wrapped_func] Took 0:00:00.320538 to load and pro

score=0.9965


' Reproduction information\n10X PBMC:\npython scdeepcluster.py --dataset 10X_PBMC\n\nMouse ES:\npython scdeepcluster.py --dataset mouse_ES_cell\n\nWorm Neuron:\npython scdeepcluster.py --dataset worm_neuron_cell --pretrain_epochs 300\n\nMouse Bladder:\npython scdeepcluster.py --dataset mouse_bladder_cell --pretrain_epochs 300 --sigma 2.75\n'

In [7]:
scdeepcluster_scores

[0.9965362702578435,
 0.24032257798854678,
 0.5352188341844301,
 0.6334722718620797,
 0.7695415409808991,
 0.8565223251023905,
 0.8565223251023905,
 0.9965362702578435]

In [8]:
import argparse

from dance.datasets.singlemodality import ClusteringDataset
from dance.modules.single_modality.clustering.scdsc import ScDSC
from dance.utils import set_seed


parser = argparse.ArgumentParser()

# model_para = [n_enc_1(n_dec_3), n_enc_2(n_dec_2), n_enc_3(n_dec_1)]
model_para = [512, 256, 256]
# Cluster_para = [n_z1, n_z2, n_z3, n_init, n_input, n_clusters]
Cluster_para = [256, 128, 32, 20, 100, 10]
# Balance_para = [binary_crossentropy_loss, ce_loss, re_loss, zinb_loss, sigma]
Balance_para = [1, 0.01, 0.1, 0.1, 1]

parser.add_argument("--data_dir", default="./data")
parser.add_argument("--dataset", type=str, default="worm_neuron_cell",
                    choices=["10X_PBMC", "mouse_bladder_cell", "mouse_ES_cell", "worm_neuron_cell","mouse_kidney_cell","human_pbmc_cell","human_pbmc2_cell","human_skin_cell","human_lung_cell","mouse_kidney_drop","mouse_kidney_cl2","mouse_kidney_10x"])
# TODO: implement callbacks for "heat_kernel" and "cosine_normalized"
parser.add_argument("--method", type=str, default="correlation", choices=["cosine", "correlation"])
parser.add_argument("--batch_size", default=256, type=int)
parser.add_argument("--n_enc_1", default=model_para[0], type=int)
parser.add_argument("--n_enc_2", default=model_para[1], type=int)
parser.add_argument("--n_enc_3", default=model_para[2], type=int)
parser.add_argument("--n_dec_1", default=model_para[2], type=int)
parser.add_argument("--n_dec_2", default=model_para[1], type=int)
parser.add_argument("--n_dec_3", default=model_para[0], type=int)
parser.add_argument("--topk", type=int, default=50)
parser.add_argument("--lr", type=float, default=1e-2)
parser.add_argument("--pretrain_lr", type=float, default=1e-3)
parser.add_argument("--pretrain_epochs", type=int, default=200)
parser.add_argument("--epochs", type=int, default=1000)
parser.add_argument("--n_z1", default=Cluster_para[0], type=int)
parser.add_argument("--n_z2", default=Cluster_para[1], type=int)
parser.add_argument("--n_z3", default=Cluster_para[2], type=int)
parser.add_argument("--n_input", type=int, default=Cluster_para[4])
parser.add_argument("--n_clusters", type=int, default=Cluster_para[5])
parser.add_argument("--device", type=str, default="cpu")
parser.add_argument("--v", type=int, default=1)
parser.add_argument("--nb_genes", type=int, default=2000)
parser.add_argument("--binary_crossentropy_loss", type=float, default=Balance_para[0])
parser.add_argument("--ce_loss", type=float, default=Balance_para[1])
parser.add_argument("--re_loss", type=float, default=Balance_para[2])
parser.add_argument("--zinb_loss", type=float, default=Balance_para[3])
parser.add_argument("--sigma", type=float, default=Balance_para[4])
parser.add_argument("--seed", type=int, default=42)
parser.add_argument("--cache", action="store_true", help="Cache processed data.")
scdsc_scores=[]
for dataset in datasets:
    args = parser.parse_args(['--dataset', dataset, '--method','cosine','--topk', '30', '--v', '7', '--binary_crossentropy_loss', '0.75','--ce_loss', '0.5', '--re_loss','0.1' ,'--zinb_loss','2.5','--sigma', '0.4'])
    set_seed(args.seed)

    # Load data and perform necessary preprocessing
    dataloader = ClusteringDataset(args.data_dir, args.dataset)
    preprocessing_pipeline = ScDSC.preprocessing_pipeline(n_top_genes=args.nb_genes, n_neighbors=args.topk)
    data = dataloader.load_data(transform=preprocessing_pipeline, cache=args.cache)

    # inputs: adj, x, x_raw, n_counts
    inputs, y = data.get_data(return_type="default")
    args.n_input = inputs[1].shape[1]

    model = ScDSC(pretrain_path=f"scdsc_{args.dataset}_pre.pkl", sigma=args.sigma, n_enc_1=args.n_enc_1,
                  n_enc_2=args.n_enc_2, n_enc_3=args.n_enc_3, n_dec_1=args.n_dec_1, n_dec_2=args.n_dec_2,
                  n_dec_3=args.n_dec_3, n_z1=args.n_z1, n_z2=args.n_z2, n_z3=args.n_z3, n_clusters=args.n_clusters,
                  n_input=args.n_input, v=args.v, device=args.device)

    # Build and train model
    model.fit(inputs, y, lr=args.lr, epochs=args.epochs, bcl=args.binary_crossentropy_loss, cl=args.ce_loss,
              rl=args.re_loss, zl=args.zinb_loss, pt_epochs=args.pretrain_epochs, pt_batch_size=args.batch_size,
              pt_lr=args.pretrain_lr)

    # Evaluate model predictions
    score = model.score(None, y)
    print(f"{score=:.4f}")
    scdsc_scores.append(score)
"""Reproduction information
10X PBMC:
python scdsc.py --dataset 10X_PBMC --method cosine --topk 30 --v 7 --binary_crossentropy_loss 0.75 --ce_loss 0.5 --re_loss 0.1 --zinb_loss 2.5 --sigma 0.4

Mouse Bladder:
python scdsc.py --dataset mouse_bladder_cell --topk 50 --v 7 --binary_crossentropy_loss 2.5 --ce_loss 0.1 --re_loss 0.5 --zinb_loss 1.5 --sigma 0.6

Mouse ES:
python scdsc.py --dataset mouse_ES_cell --topk 50 --v 7 --binary_crossentropy_loss 0.1 --ce_loss 0.01 --re_loss 1.5 --zinb_loss 0.5 --sigma 0.1

Worm Neuron:
python scdsc.py --dataset worm_neuron_cell --topk 20 --v 7 --binary_crossentropy_loss 2 --ce_loss 2 --re_loss 3 --zinb_loss 0.1 --sigma 0.4
"""


[INFO][2023-09-02 16:00:57,799][dance][set_seed] Setting global random seed to 42


[INFO][2023-09-02 16:00:57,887][dance][load_data] Raw data loaded:
Data object that wraps (.data):
AnnData object with n_obs × n_vars = 902 × 16468
    uns: 'dance_config'
    obsm: 'Group'
[INFO][2023-09-02 16:00:57,888][dance.Compose][__call__] Applying composed transformations:
Compose(
  AnnDataTransform(func=scanpy.preprocessing._simple.filter_genes, func_kwargs={'min_counts': 3}),
  AnnDataTransform(func=scanpy.preprocessing._simple.filter_cells, func_kwargs={'min_counts': 1}),
  AnnDataTransform(func=scanpy.preprocessing._simple.normalize_per_cell, func_kwargs={}),
  AnnDataTransform(func=scanpy.preprocessing._simple.log1p, func_kwargs={}),
  AnnDataTransform(func=scanpy.preprocessing._highly_variable_genes.highly_variable_genes, func_kwargs={'min_mean': 0.0125, 'max_mean': 4, 'flavor': 'cell_ranger', 'min_disp': 0.5, 'n_top_genes': 2000, 'subset': True}),
  AnnDataTransform(func=scanpy.preprocessing._simple.filter_genes, func_kwargs={'min_counts': 1}),
  AnnDataTransform(func=s

[INFO][2023-09-02 16:00:58,168][dance.NeighborGraph][__call__] Start computing the kNN connectivity adjacency matrix
[INFO][2023-09-02 16:01:00,596][dance.SetConfig][__call__] Updating the dance data object config options:
{'feature_channel': ['NeighborGraph', None, None, 'n_counts'],
 'feature_channel_type': ['obsp', 'X', 'raw_X', 'obs'],
 'label_channel': 'Group'}
[INFO][2023-09-02 16:01:00,596][dance][set_config_from_dict] Setting config 'feature_channel' to ['NeighborGraph', None, None, 'n_counts']
[INFO][2023-09-02 16:01:00,597][dance][set_config_from_dict] Setting config 'feature_channel_type' to ['obsp', 'X', 'raw_X', 'obs']
[INFO][2023-09-02 16:01:00,598][dance][set_config_from_dict] Setting config 'label_channel' to 'Group'
[INFO][2023-09-02 16:01:00,599][dance][load_data] Data transformed:
Data object that wraps (.data):
AnnData object with n_obs × n_vars = 902 × 2000
    obs: 'n_counts'
    var: 'n_counts', 'highly_variable', 'means', 'dispersions', 'dispersions_norm', 'mean

score=0.5722


[INFO][2023-09-02 16:02:04,636][dance][load_data] Raw data loaded:
Data object that wraps (.data):
AnnData object with n_obs × n_vars = 648 × 64535
    uns: 'dance_config'
    obsm: 'Group'
[INFO][2023-09-02 16:02:04,636][dance.Compose][__call__] Applying composed transformations:
Compose(
  AnnDataTransform(func=scanpy.preprocessing._simple.filter_genes, func_kwargs={'min_counts': 3}),
  AnnDataTransform(func=scanpy.preprocessing._simple.filter_cells, func_kwargs={'min_counts': 1}),
  AnnDataTransform(func=scanpy.preprocessing._simple.normalize_per_cell, func_kwargs={}),
  AnnDataTransform(func=scanpy.preprocessing._simple.log1p, func_kwargs={}),
  AnnDataTransform(func=scanpy.preprocessing._highly_variable_genes.highly_variable_genes, func_kwargs={'min_mean': 0.0125, 'max_mean': 4, 'flavor': 'cell_ranger', 'min_disp': 0.5, 'n_top_genes': 2000, 'subset': True}),
  AnnDataTransform(func=scanpy.preprocessing._simple.filter_genes, func_kwargs={'min_counts': 1}),
  AnnDataTransform(func=s

...

score=0.3271


[INFO][2023-09-02 16:17:27,510][dance.SaveRaw][__call__] Saving data to ``.raw``


[INFO][2023-09-02 16:17:27,521][dance.NeighborGraph][__call__] Start computing the kNN connectivity adjacency matrix
[INFO][2023-09-02 16:17:28,104][dance.SetConfig][__call__] Updating the dance data object config options:
{'feature_channel': ['NeighborGraph', None, None, 'n_counts'],
 'feature_channel_type': ['obsp', 'X', 'raw_X', 'obs'],
 'label_channel': 'Group'}
[INFO][2023-09-02 16:17:28,105][dance][set_config_from_dict] Setting config 'feature_channel' to ['NeighborGraph', None, None, 'n_counts']
[INFO][2023-09-02 16:17:28,106][dance][set_config_from_dict] Setting config 'feature_channel_type' to ['obsp', 'X', 'raw_X', 'obs']
[INFO][2023-09-02 16:17:28,107][dance][set_config_from_dict] Setting config 'label_channel' to 'Group'
[INFO][2023-09-02 16:17:28,108][dance][load_data] Data transformed:
Data object that wraps (.data):
AnnData object with n_obs × n_vars = 902 × 2000
    obs: 'n_counts'
    var: 'n_counts', 'highly_variable', 'means', 'dispersions', 'dispersions_norm', 'mean

score=0.5907


'Reproduction information\n10X PBMC:\npython scdsc.py --dataset 10X_PBMC --method cosine --topk 30 --v 7 --binary_crossentropy_loss 0.75 --ce_loss 0.5 --re_loss 0.1 --zinb_loss 2.5 --sigma 0.4\n\nMouse Bladder:\npython scdsc.py --dataset mouse_bladder_cell --topk 50 --v 7 --binary_crossentropy_loss 2.5 --ce_loss 0.1 --re_loss 0.5 --zinb_loss 1.5 --sigma 0.6\n\nMouse ES:\npython scdsc.py --dataset mouse_ES_cell --topk 50 --v 7 --binary_crossentropy_loss 0.1 --ce_loss 0.01 --re_loss 1.5 --zinb_loss 0.5 --sigma 0.1\n\nWorm Neuron:\npython scdsc.py --dataset worm_neuron_cell --topk 20 --v 7 --binary_crossentropy_loss 2 --ce_loss 2 --re_loss 3 --zinb_loss 0.1 --sigma 0.4\n'

In [9]:
scdsc_scores

[0.5722479940804349,
 0.21680167115856036,
 0.32343149332579424,
 0.5530234095994098,
 0.6596624201855319,
 0.3270840856044606,
 0.3270840856044606,
 0.5907197301775053]

In [10]:
import torch
torch.cuda.is_available()

True

In [11]:
import argparse

import numpy as np

from dance.datasets.singlemodality import ClusteringDataset
from dance.modules.single_modality.clustering.sctag import ScTAG
from dance.utils import set_seed

parser = argparse.ArgumentParser(description="train", formatter_class=argparse.ArgumentDefaultsHelpFormatter)
parser.add_argument("--data_dir", default="./data", type=str)
parser.add_argument("--dataset", default="mouse_bladder_cell", type=str,
                    choices=["10X_PBMC", "mouse_bladder_cell", "mouse_ES_cell", "worm_neuron_cell","mouse_kidney_cell","human_pbmc_cell","human_pbmc2_cell","human_skin_cell","human_lung_cell","mouse_kidney_drop","mouse_kidney_cl2","mouse_kidney_10x"])
parser.add_argument("--k_neighbor", default=15, type=int)
parser.add_argument("--highly_genes", default=3000, type=int)
parser.add_argument("--pca_dim", default=50, type=int)
parser.add_argument("--k", default=2, type=int)
parser.add_argument("--hidden_dim", default=128, type=int)
parser.add_argument("--latent_dim", default=15, type=int)
parser.add_argument("--dec_dim", default=None, type=int)
parser.add_argument("--dropout", default=0.4, type=float)
parser.add_argument("--alpha", default=1.0, type=float)
parser.add_argument("--pretrain_epochs", default=200, type=int)
parser.add_argument("--epochs", default=500, type=int)
parser.add_argument("--device", default="cpu")
parser.add_argument("--w_a", default=1, type=float)
parser.add_argument("--w_x", default=1, type=float)
parser.add_argument("--w_d", default=0, type=float)
parser.add_argument("--w_c", default=1, type=float)
parser.add_argument("--lr", default=1e-3, type=float)
parser.add_argument("--min_dist", default=0.5, type=float)
parser.add_argument("--max_dist", default=20.0, type=float)
parser.add_argument("--info_step", default=50, type=int)
parser.add_argument("--seed", type=int, default=42)
parser.add_argument("--cache", action="store_true", help="Cache processed data.")
sctag_scores=[]
for dataset in datasets:
    args = parser.parse_args([ '--dataset',dataset,'--pretrain_epochs','100',"--k","1"])
    set_seed(args.seed)

    # Load data and perform necessary preprocessing
    dataloader = ClusteringDataset(args.data_dir, args.dataset)
    preprocessing_pipeline = ScTAG.preprocessing_pipeline(n_top_genes=args.highly_genes, n_components=args.pca_dim,
                                                          n_neighbors=args.k_neighbor)
    data = dataloader.load_data(transform=preprocessing_pipeline, cache=args.cache)

    # inputs: adj, x, x_raw, n_counts
    inputs, y = data.get_train_data()

    n_clusters = len(np.unique(y))

    # Build and train model
    model = ScTAG(n_clusters=n_clusters, k=args.k, hidden_dim=args.hidden_dim, latent_dim=args.latent_dim,
                  dec_dim=args.dec_dim, dropout=args.dropout, device=args.device, alpha=args.alpha,
                  pretrain_path=f"sctag_{args.dataset}_pre.pkl")
    print(np.max(inputs[0]))
    model.fit(inputs, y, epochs=args.epochs, pretrain_epochs=args.pretrain_epochs, lr=args.lr, w_a=args.w_a,
              w_x=args.w_x, w_c=args.w_c, w_d=args.w_d, info_step=args.info_step, max_dist=args.max_dist,
              min_dist=args.min_dist)

    # Evaluate model predictions
    score = model.score(None, y)
    print(f"{score=:.4f}")
    sctag_scores.append(score)
"""Reproduction information
10X PBMC:
python sctag.py --dataset 10X_PBMC --pretrain_epochs 100 --w_a 0.01 --w_x 3 --w_c 0.1 --dropout 0.5

Mouse ES:
python sctag.py --dataset mouse_ES_cell --pretrain_epochs 100 --w_a 0.01 --w_x 0.75 --w_c 1

Worm Neuron:
python sctag.py --dataset worm_neuron_cell --w_a 0.01 --w_x 2 --w_c 0.25 --k 1

Mouse Bladder:
python sctag.py --dataset mouse_bladder_cell --pretrain_epochs 100 --w_a 0.1 --w_x 2.5 --w_c 3
"""


[INFO][2023-09-02 16:18:32,108][dance][set_seed] Setting global random seed to 42
[INFO][2023-09-02 16:18:32,181][dance][load_data] Raw data loaded:
Data object that wraps (.data):
AnnData object with n_obs × n_vars = 902 × 16468
    uns: 'dance_config'
    obsm: 'Group'
[INFO][2023-09-02 16:18:32,181][dance.Compose][__call__] Applying composed transformations:
Compose(
  AnnDataTransform(func=scanpy.preprocessing._simple.filter_genes, func_kwargs={'min_counts': 3}),
  AnnDataTransform(func=scanpy.preprocessing._simple.filter_cells, func_kwargs={'min_counts': 1}),
  AnnDataTransform(func=scanpy.preprocessing._simple.normalize_per_cell, func_kwargs={}),
  AnnDataTransform(func=scanpy.preprocessing._simple.log1p, func_kwargs={}),
  AnnDataTransform(func=scanpy.preprocessing._highly_variable_genes.highly_variable_genes, func_kwargs={'min_mean': 0.0125, 'max_mean': 4, 'flavor': 'cell_ranger', 'min_disp': 0.5, 'n_top_genes': 3000, 'subset': True}),
  AnnDataTransform(func=scanpy.preprocessi

[INFO][2023-09-02 16:18:32,491][dance.CellPCA][__call__] Start generating cell PCA features (902, 3000) (k=50)
[INFO][2023-09-02 16:18:32,527][dance.CellPCA][__call__] Top 10 explained variances: [0.17064218 0.09295565 0.03068668 0.02320614 0.01286646 0.0118582
 0.00996639 0.00687872 0.0059774  0.00535993]
[INFO][2023-09-02 16:18:32,527][dance.CellPCA][__call__] Total explained variance: 47.30%
[INFO][2023-09-02 16:18:32,528][dance.NeighborGraph][__call__] Start computing the kNN connectivity adjacency matrix
[INFO][2023-09-02 16:18:32,570][dance.SetConfig][__call__] Updating the dance data object config options:
{'feature_channel': ['NeighborGraph', None, None, 'n_counts'],
 'feature_channel_type': ['obsp', 'X', 'raw_X', 'obs'],
 'label_channel': 'Group'}
[INFO][2023-09-02 16:18:32,570][dance][set_config_from_dict] Setting config 'feature_channel' to ['NeighborGraph', None, None, 'n_counts']
[INFO][2023-09-02 16:18:32,571][dance][set_config_from_dict] Setting config 'feature_channel_t

1.0


/home/zyxing/anaconda3/envs/dance-env/lib/python3.8/site-packages/dgl/backend/pytorch/tensor.py:352: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  assert input.numel() == input.storage().size(), "Cannot convert view " \
[INFO][2023-09-02 16:18:32,727][dance][fit] Epoch   1, ARI: 0.9965, Best ARI: 0.9965
/home/zyxing/anaconda3/envs/dance-env/lib/python3.8/site-packages/dgl/backend/pytorch/tensor.py:352: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  assert input.numel() == input.storage().size(), "Cannot convert vi

score=1.0000


...

1.0


/home/zyxing/anaconda3/envs/dance-env/lib/python3.8/site-packages/dgl/backend/pytorch/tensor.py:352: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  assert input.numel() == input.storage().size(), "Cannot convert view " \
[INFO][2023-09-02 16:32:11,247][dance][fit] Epoch   1, ARI: 0.9965, Best ARI: 0.9965
/home/zyxing/anaconda3/envs/dance-env/lib/python3.8/site-packages/dgl/backend/pytorch/tensor.py:352: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  assert input.numel() == input.storage().size(), "Cannot convert vi

score=1.0000


/home/zyxing/anaconda3/envs/dance-env/lib/python3.8/site-packages/dgl/backend/pytorch/tensor.py:352: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  assert input.numel() == input.storage().size(), "Cannot convert view " \


'Reproduction information\n10X PBMC:\npython sctag.py --dataset 10X_PBMC --pretrain_epochs 100 --w_a 0.01 --w_x 3 --w_c 0.1 --dropout 0.5\n\nMouse ES:\npython sctag.py --dataset mouse_ES_cell --pretrain_epochs 100 --w_a 0.01 --w_x 0.75 --w_c 1\n\nWorm Neuron:\npython sctag.py --dataset worm_neuron_cell --w_a 0.01 --w_x 2 --w_c 0.25 --k 1\n\nMouse Bladder:\npython sctag.py --dataset mouse_bladder_cell --pretrain_epochs 100 --w_a 0.1 --w_x 2.5 --w_c 3\n'

In [12]:
sctag_scores

[1.0,
 0.8596006841137562,
 0.40078924676001404,
 0.9156527237548763,
 0.8447362442419666,
 0.9209258226474065,
 0.9209258226474065,
 1.0]